In [2]:
# standard imports
import numpy as np
import os
import csv
from tqdm import tqdm
import meshio
import matplotlib.pyplot as plt

In [3]:
# custom class imports
from node import node
from cell import cell
from mesh import exp_mesh
from derivatives import *
from fluid_problem import isentropic_fluid_problem

# utility functions
from utilities_modified import *
from isentropic_navier_stokes import Q,L,inner_product

# Setting Problem

In [4]:
# read su2 mesh file
meshio_mesh = meshio.read("./lam_cyl_comp_air/mesh_cylinder_quad.su2",file_format="su2")
# define mid point for rotational symmetric mesh
mid_point = np.array([0.5,0]) 
# convert mesh into custom class exp_mesh
mesh = exp_mesh(meshio_mesh,mid_point)

    Surface tag  cylinder will be replaced by 1
    Surface tag  farfield will be replaced by 2
-----------------------Custom mesh initialization started-----------------------
------------Shifted and Cylinder coordinates calculated and stored!-------------
-------------------Cell volumes computed in cell structures!--------------------
-------------------Cell centers computed in cell structures!--------------------
----------------------------Node neighbors assigned!----------------------------
----------------------Nodal volume participation computed!----------------------
---------------------Custom mesh initialized successfully!----------------------


# Loading Filedata

In [25]:
# reading file data
data_dir="./lam_cyl_comp_air/mach 0.01/"
file_name_structure = "restart_flow_"
data_dict = read_csv_data(data_dir,file_name_structure,delay=5000,subsampling=2,max_size=180)

100%|██████████| 180/180 [00:59<00:00,  3.00it/s]


# Data Construction

In [26]:
data_dict.keys()

dict_keys(['PointID', 'x', 'y', 'Density', 'Momentum_x', 'Momentum_y', 'Energy', 'Pressure', 'Temperature', 'Mach', 'Pressure_Coefficient', 'Laminar_Viscosity', 'Skin_Friction_Coefficient_x', 'Skin_Friction_Coefficient_y', 'Heat_Flux', 'Y_Plus'])

In [27]:
x = data_dict["x"][:,0] - 0.5
y = data_dict["y"][:,0]
u = np.divide(data_dict['Momentum_x'],data_dict['Density'])
v = np.divide(data_dict['Momentum_y'],data_dict['Density'])
a = data_dict['Mach']
n = len(x)

# POD Decomposition

In [28]:
# configure state data 
q = np.vstack([u,v,a])
q_avg = np.mean(q,1)
q_fluc = q - np.repeat(np.expand_dims(q_avg,1),q.shape[1],axis=1)

### Euclidean POD Modes

In [29]:
U, S, Vh = np.linalg.svd(q_fluc, full_matrices=False)

# reconstruction threshold
epsilon = 0.97 # [0,1]
acc = 0
num = 0
while acc < epsilon:
    num+=1
    acc = sum(S[:num])/sum(S)    
print("Number of eigenvectors necessary for thresholded reconstruction: \t" + str(num))
if num % 2 ==1:
    num+=1

coeffs_eucl = np.matmul(np.transpose(U[:,:num]),q_fluc)
pod_modes_eucl = U

Number of eigenvectors necessary for thresholded reconstruction: 	4


### Isentropic POD Modes

In [30]:
T = q.shape[1]
C = np.empty((T,T))
alpha = 1
for i in tqdm(range(T)):
    for j in range(i,T):
        C[i,j] = (1/T) * inner_product(mesh,q_fluc[:,i],q_fluc[:,j],alpha)
        C[j,i] = C[i,j]

100%|██████████| 180/180 [09:12<00:00,  3.07s/it]


In [31]:
S,V =  np.linalg.eig(C)

epsilon = 0.97 # [0,1]
acc = 0
num = 0
while acc < epsilon:
    num+=1
    acc = sum(S[:num])/sum(S)    
print("Number of eigenvectors necessary for thresholded reconstruction: \t" + str(num))
if num % 2 == 1:
    num+=1
num = 6

Number of eigenvectors necessary for thresholded reconstruction: 	2


In [32]:
# construct spatial POD Modes - not normalized
pod_modes = np.zeros((q.shape[0],q.shape[1]))
for i in range(len(S)):
    pod_modes += np.outer(q_fluc[:,i],V[i,:])

# normalizing POD Modes
for i in range(len(S)):
    pod_modes[:,i]/=np.sqrt(inner_product(mesh,pod_modes[:,i],pod_modes[:,i],alpha))

# Solve for Optimal Activations

In [33]:
coeffs = np.matmul(np.linalg.pinv(pod_modes),q_fluc) # isentropic
coeffs_eucl = coeffs_eucl # euclidean 

In [47]:
# plotting activations
def plot_activations(coeffs):
    dt = 1 
    t = np.linspace(0,(coeffs.shape[1]-1)*dt,coeffs.shape[1])

    rows = int(np.ceil(num/2))
    fig,ax = plt.subplots(rows,2,figsize=(7.5*rows,10))
    fig.tight_layout(pad=3.0)
    if rows==1:
        ax[0].plot(t,coeffs[0,:])
        ax[0].title.set_text("Activation for Eigenflow 0")
        ax[1].plot(t,coeffs[1,:])
        ax[1].title.set_text("Activation for Eigenflow 1")
    else:
        for i in range(rows):
            print(i)
            ax[i][0].plot(t,coeffs[2*i,:])
            ax[i][0].title.set_text("Activation for Eigenflow " + str(2*i))
            ax[i][1].plot(t,coeffs[2*i+1,:])
            ax[i][1].title.set_text("Activation for Eigenflow " + str(2*i+1))

In [48]:
plot_activations(coeffs_eucl)

3
0
1
2


IndexError: index 4 is out of bounds for axis 0 with size 4